### Dataset: https://www.kaggle.com/datasets/andrewkronser/cve-common-vulnerabilities-and-exposures

## KMeans: https://realpython.com/k-means-clustering-python/

In [ ]:
!nvidia-smi

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!ls "/content/drive/MyDrive/MachineLearning/archive.zip"
!unzip "/content/drive/MyDrive/MachineLearning/archive.zip"

ls: cannot access '/content/drive/MyDrive/MachineLearning/archive.zip': No such file or directory
unzip:  cannot find or open /content/drive/MyDrive/MachineLearning/archive.zip, /content/drive/MyDrive/MachineLearning/archive.zip.zip or /content/drive/MyDrive/MachineLearning/archive.zip.ZIP.


In [ ]:
!pip show accelerate
!pip install accelerate -U

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
cve = pd.read_csv('/content/cve.csv')

In [ ]:
!pip show transformers

In [ ]:
!pip install transformers datasets evaluate

In [ ]:
from transformers import pipeline

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

In [ ]:
def preprocess_function(example):
    token = tokenizer(example, truncation=True)
    return token

In [ ]:
X = cve['summary'].tolist()
y = cve['cwe_name'].tolist()

In [ ]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()

In [ ]:
cve['cwe_name_code'] = label_encoder.fit_transform(cve['cwe_name'])

In [ ]:
y_numeric = cve['cwe_name_code'].tolist()

In [ ]:
tokenized_cve = list(map(preprocess_function, X))

In [ ]:
print(tokenized_cve[0].keys())

tokenized_cve_input_ids_as_list = []
tokenized_cve_attention_mask_as_list = []
tokenized_cve_text_as_list = []
tokenized_cve_label_as_list = []

for i in range(len(tokenized_cve)):
  tokenized_cve[i]['text'] = X[i]
  tokenized_cve[i]['label'] = y_numeric[i]

  tokenized_cve_input_ids_as_list.append(tokenized_cve[i]['input_ids'])
  tokenized_cve_attention_mask_as_list.append(tokenized_cve[i]['attention_mask'])
  tokenized_cve_text_as_list.append(tokenized_cve[i]['text'])
  tokenized_cve_label_as_list.append(tokenized_cve[i]['label'])

dict_keys(['input_ids', 'attention_mask'])


In [ ]:
max(y_numeric)

169

In [ ]:
d = {
  'text': tokenized_cve_text_as_list,
  'label': tokenized_cve_label_as_list,
  'input_ids': tokenized_cve_input_ids_as_list,
  'attention_mask': tokenized_cve_attention_mask_as_list,
}
df = pd.DataFrame(data=d)

from datasets import Dataset
import pyarrow as pa

pa_arrow = pa.Table.from_pandas(df)
tokenized_cve_ds = Dataset(pa_arrow)

print(tokenized_cve_ds)

Dataset({
    features: ['text', 'label', 'input_ids', 'attention_mask'],
    num_rows: 89660
})


In [ ]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
import evaluate

accuracy = evaluate.load("accuracy")

In [ ]:
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [ ]:
'''
id2label = {
    0: "NEGATIVE",
    1: "POSITIVE"
}

label2id = {
    "NEGATIVE": 0,
    "POSITIVE": 1
}
'''
id2label = {}
label2id = {}

cwe_name_list = cve['cwe_name'].tolist()
cwe_name_code_list = cve['cwe_name_code'].tolist()

for x in range(len(cwe_name_list)):
  id2label[cwe_name_code_list[x]] = cwe_name_list[x]
  label2id[cwe_name_list[x]] = cwe_name_code_list[x]

In [ ]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased",
    num_labels=170,
    id2label=id2label,
    label2id=label2id
)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
training_args = TrainingArguments(
    output_dir="my_awesome_model",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=False,
)

In [ ]:
# https://huggingface.co/docs/transformers/main_classes/callback

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_cve_ds,
    eval_dataset=tokenized_cve_ds,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Accuracy
1,0.968100,0.870985,0.778497
2,0.852100,0.741540,0.805800


TrainOutput(global_step=11208, training_loss=1.0599419051285388, metrics={'train_runtime': 3816.6935, 'train_samples_per_second': 46.983, 'train_steps_per_second': 2.937, 'total_flos': 8664315095751120.0, 'train_loss': 1.0599419051285388, 'epoch': 2.0})

In [ ]:
print(type(model))
print(type(tokenizer))

<class 'transformers.models.distilbert.modeling_distilbert.DistilBertForSequenceClassification'>
<class 'transformers.models.distilbert.tokenization_distilbert_fast.DistilBertTokenizerFast'>


In [ ]:
model.save_pretrained('/content/drive/MyDrive/MachineLearning/model_trained_04_08')
tokenizer.save_pretrained('/content/drive/MyDrive/MachineLearning/tokenizer_trained_04_08')

('/content/drive/MyDrive/MachineLearning/tokenizer_trained_04_08/tokenizer_config.json',
 '/content/drive/MyDrive/MachineLearning/tokenizer_trained_04_08/special_tokens_map.json',
 '/content/drive/MyDrive/MachineLearning/tokenizer_trained_04_08/vocab.txt',
 '/content/drive/MyDrive/MachineLearning/tokenizer_trained_04_08/added_tokens.json',
 '/content/drive/MyDrive/MachineLearning/tokenizer_trained_04_08/tokenizer.json')

In [ ]:
tokenizer_2 = AutoTokenizer.from_pretrained('/content/drive/MyDrive/MachineLearning/tokenizer_trained_04_08')
model_2 = AutoModelForSequenceClassification.from_pretrained('/content/drive/MyDrive/MachineLearning/model_trained_04_08')

In [ ]:
print(type(model_2))
print(type(tokenizer_2))

<class 'transformers.models.distilbert.modeling_distilbert.DistilBertForSequenceClassification'>
<class 'transformers.models.distilbert.tokenization_distilbert_fast.DistilBertTokenizerFast'>


In [ ]:
# inference
text = "'Zoom for Windows clients before version 5.13.3, Zoom Rooms for Windows clients before version 5.13.5 and Zoom VDI for Windows clients before 5.13.1 contain an information disclosure vulnerability. A recent update to the Microsoft Edge WebView2 runtime used by the affected Zoom clients, transmitted text to Microsoft&#8217;s online Spellcheck service instead of the local Windows Spellcheck. Updating Zoom remediates this vulnerability by disabling the feature. Updating Microsoft Edge WebView2 Runtime to at least version 109.0.1481.0 and restarting Zoom remediates this vulnerability by updating Microsoft&#8217;s telemetry behavior."

In [ ]:
inputs = tokenizer(text, return_tensors="pt")

In [ ]:
inputs.to('cuda')

In [ ]:
import torch
with torch.no_grad():
    logits = model(**inputs).logits

In [ ]:
print(logits)

In [ ]:
predicted_class_id = logits.argmax().item()
model.config.id2label[predicted_class_id]

' Reachable Assertion'

-----------------------------------------------------------------------------------

In [ ]:
inputs = tokenizer_2(text, return_tensors="pt")

In [ ]:
inputs.to('cuda')
model_2.to('cuda')

In [ ]:
import torch
with torch.no_grad():
    logits = model_2(**inputs).logits

In [ ]:
predicted_class_id = logits.argmax().item()
model.config.id2label[predicted_class_id]

' Improper Authentication'

In [ ]:
!wmic

/bin/bash: line 1: wmic: command not found


In [ ]:
!uname -r
!lsb_release -a